In [412]:
# Install and import libraries

In [564]:
!pip install lightgbm

In [566]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os


from pathlib import Path
from sklearn.metrics import r2_score, mean_squared_error
from sklearn import model_selection
from lightgbm import LGBMRegressor

In [541]:
#Read the Dataset

In [567]:
train_path = Path().cwd().parent / 'User'/'Documents'/'Sirius'/'data' / 'train.csv'
train_df = pd.read_csv(train_path)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
train_df.head()

,datetime,Номер скважины,Дебит нефти,Давление забойное,x,y,Объем жидкости,Объем нефти,Активная мощность (ТМ),Время работы (ТМ),Газовый фактор рабочий (ТМ),Давление буферное,Давление забойное от Hд,Давление забойное от Pпр,Давление линейное (ТМ),Давление на входе ЭЦН (ТМ),Дебит газа (ТМ),Дебит газа попутного,Дебит жидкости (ТМ),Коэффициент мощности (ТМ)
0,1990-08-01,0,19.94,NaN,18670.86,5714.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1990-08-02,0,19.94,39.06,18670.86,5714.86,NaN,NaN,50.85,9.60,NaN,NaN,33.91,39.06,1.89,30.29,3843.75,24.25,24.25,98.53
2,1990-08-03,0,21.17,39.06,18670.86,5714.86,NaN,NaN,52.35,NaN,NaN,NaN,33.81,39.06,1.89,30.26,3900.95,25.75,25.83,99.14
3,1990-08-04,0,22.53,38.97,18670.86,5714.86,NaN,NaN,51.24,9.60,NaN,NaN,33.70,38.97,1.88,30.21,3874.51,27.40,27.93,98.74
4,1990-08-05,0,22.53,38.77,18670.86,5714.86,NaN,NaN,50.91,9.56,NaN,NaN,33.70,38.77,1.87,30.11,3853.70,27.40,27.40,98.42


In [568]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67136 entries, 0 to 67135
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   datetime                     67136 non-null  object 
 1   Номер скважины               67136 non-null  int64  
 2   Дебит нефти                  67136 non-null  float64
 3   Давление забойное            64756 non-null  float64
 4   x                            67136 non-null  float64
 5   y                            67136 non-null  float64
 6   Объем жидкости               3465 non-null   float64
 7   Объем нефти                  3465 non-null   float64
 8   Активная мощность (ТМ)       62412 non-null  float64
 9   Время работы (ТМ)            63475 non-null  float64
 10  Газовый фактор рабочий (ТМ)  25230 non-null  float64
 11  Давление буферное            10208 non-null  float64
 12  Давление забойное от Hд      65710 non-null  float64
 13  Давление забойно

In [569]:
#Build the Model (Split the dataset)

In [570]:
#fill nan based on well numbers via average values
well_numbers = list(sorted(set(train_df['Номер скважины'])))
for i in well_numbers:
    train_df.loc[train_df['Номер скважины'] == i] = train_df.loc[train_df['Номер скважины'] == i].fillna(train_df.loc[train_df['Номер скважины'] == i].mean())

C:\Users\User\AppData\Local\Temp\ipykernel_25452\4039835906.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_df.loc[train_df['Номер скважины'] == i] = train_df.loc[train_df['Номер скважины'] == i].fillna(train_df.loc[train_df['Номер скважины'] == i].mean())


In [571]:
y_train, y_test = [train_df[train_df['Номер скважины'] == i]['Дебит нефти'].iloc[-180:-90] for i in well_numbers], [train_df[train_df['Номер скважины'] == i]['Дебит нефти'].iloc[-90:] for i in well_numbers]

In [572]:
# Model setup and train predictions

In [573]:
results = []
for i in range(int(len(y_test))):
    model = LGBMRegressor()
    model.fit(np.array(y_train[i]).reshape(-1, 1), np.array(y_test[i]).reshape(-1, 1).ravel())
    results.append(model.predict(np.array(y_train[i]).reshape(-1, 1)))

In [574]:
#Train prediction

In [575]:
def result_plot(y_pred, y_real, n_points=90):
    RMSE, R2 = [], []
    
    RMSE.append(np.sqrt(mean_squared_error(y_real, y_pred)))
    R2.append(r2_score(y_real, y_pred))
    
    return RMSE[0], R2[0]

In [576]:
RMSE_list = []
R2_list = []
for i in range(len(y_test)):
    temp = result_plot(results[50], y_test[50])
    RMSE_list.append(temp[0])
    R2_list.append(temp[-1])
print(f'Train RMSE: {np.mean(RMSE_list)}')
print(f'Train R2: {np.mean(R2_list)}')

Train RMSE: 1.4313829906029942
Train R2: 0.27551141479972774


In [577]:
#Test prediction

In [578]:
X_test = [train_df[train_df['Номер скважины'] == i]['Дебит нефти'].iloc[-90:] for i in well_numbers]

In [579]:
results_df = []
date_range = pd.date_range(start='1992-04-11', freq='1D', periods=90)
for i in range(int(len(X_test))):
    model = LGBMRegressor()
    model.fit(np.array(y_train[i]).reshape(-1, 1), np.array(y_test[i]).reshape(-1, 1).ravel())
    prediction_temp = model.predict(np.array(X_test[i]).reshape(-1, 1))
    forecast_df = pd.DataFrame({'datetime': date_range, 'forecast': prediction_temp, 'Номер скважины':well_numbers[i]})
    results_df.append(forecast_df)

In [580]:
#Save results

In [581]:
solution_df = pd.concat(results_df)

In [582]:
solution_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9540 entries, 0 to 89
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   datetime        9540 non-null   datetime64[ns]
 1   forecast        9540 non-null   float64       
 2   Номер скважины  9540 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 298.1 KB


In [584]:
solution_df.to_csv('Solution3.csv', index=False, encoding="utf-8")
print('Solution3.csv')

Solution3.csv
